In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim


from torch.autograd import Variable

In [2]:
data_df = pd.DataFrame({"x1": np.arange(1, 20),
                        "x2": 10*np.sin(np.arange(1, 20))})
data_df['y'] = 3*data_df["x1"] + 2 * data_df["x2"] + np.random.randn(data_df.shape[0])
data_df.head()

,x1,x2,y
0,1,8.414710,19.842122
1,2,9.092974,24.514249
2,3,1.411200,10.798747
3,4,-7.568025,-3.443418
4,5,-9.589243,-4.800045


$$y = 3x_1 + 2x_2 + \epsilon$$

In [3]:
# lots of code taken from 
# https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817


class LinearRegression(nn.Module):
    
    def __init__(self, 
                 criterion=None,
                 optimizer="adam",
                 epochs=200):
        super(LinearRegression, self).__init__()
        
        self.optimizer_name = optimizer
        
        self.criterion = criterion
        if (self.criterion is None):
            self.criterion = nn.MSELoss()
        self.epochs = epochs
    
        
    def forward(self, X):
        out = self.linear(X)
        return out.view(-1, 1)
    
    def backward(self, X, y):
        inputs = torch.from_numpy(X)
        labels = torch.from_numpy(y).view(-1,1)
        self.optim.zero_grad()
        
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        
        loss.backward()
        self.optim.step()
        
        return loss
    
    def fit(self, X, y):
        #X = np.c_[np.ones(X.shape[0]), X].astype(np.float32)
        
        # if X and y are pandas df
        X = X.values.astype(np.float32)
        y = y.values.astype(np.float32)
        
        
        self.feature_number = X.shape[1]
        self.target_number = 1 if len(y.shape) == 1 else y.shape[1]
        
        
        self.linear = nn.Linear(self.feature_number, self.target_number)
        
        if (self.optimizer_name == "adam"):
            self.optim = optim.Adam(self.parameters(), lr=0.1)
        else:
            raise ValueError(f'Optimizer {self.optimizer_name} is not recognized')
        
        
        for epoch in range(self.epochs):
            self.backward(X, y)
            
    @property
    def coef_(self):
        
        return self.linear.weight.detach().numpy()
    
    @property
    def const_(self):
        return self.linear.bias.detach().numpy()
    
    

In [4]:
model = LinearRegression(epochs=2000)

In [5]:
X = data_df.iloc[:, :-1]
y = data_df.iloc[:, -1]

model.fit(X,y)

In [6]:
model.coef_

array([[2.9957278, 2.0589032]], dtype=float32)

In [7]:
model.const_

array([-0.10196462], dtype=float32)

In [ ]:
from sklearn.linear_model import LinearRegression

my_lr = LinearRegression()
my_lr.fit(X,y)
print(my_lr.coef_)
print(my_lr.intercept_)